In [98]:
import sqlite3
import pandas as pd

In [99]:
con = sqlite3.connect(":memory:")
cur = con.cursor()

https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html

In [100]:
data = pd.read_csv('boston.csv')

In [101]:
data.to_sql("boston", con, index=False, if_exists="replace")

506

In [102]:
pd.read_sql(
    """
    SELECT * FROM boston
    limit 5
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### Все задания нужно выполнить только с помощью sql

1. Выведите количество пустых значений по колонкам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во пустых значений)

In [103]:
pd.read_sql(
    """
    select 
    count (*) filter (where CRIM=0.0) as CRIM,
    count(*) filter (where zn=0.0) as ZN,
    count (*) filter (where INDUS=0.0) as INDUS,
    count (*) filter (where CHAS=0.0) as CHAS,
    count (*) filter (where NOX=0.0) as NOX from boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,0,372,0,471,0


2. Выведите количество уникальных значений по колонокам CRIM, ZN, INDUS, CHAS, NOX (название колонки, кол-во уникальных значений)

In [104]:
pd.read_sql(
    """
    select count(distinct CRIM) filter (where CRIM!=0.0) as CRIM,
    count (distinct ZN) filter (where ZN!=0.0) as ZN,
    count (distinct INDUS) filter (where INDUS!=0.0) as INDUS,
    count (distinct CHAS) filter (where CHAS!=0.0) as CHAS,
    count (distinct NOX) filter (where NOX!=0.0) as NOX
    from boston
    """,
    con,
)

,CRIM,ZN,INDUS,CHAS,NOX
0,504,25,76,1,81


3. Выведите колонки, у которых медиана равна минимальному значению (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX.
Напишите какой вывод можно сделать по данным в этих колонках

In [105]:
pd.read_sql(
    """
    with medians as (
    select
    nth_value(CRIM, (select count(*)/2 from boston)) over (order by CRIM) as CRIM_MEDIAN,
    min(CRIM) over () as MIN_CRIM,
    nth_value(ZN, (select count(*)/2 from boston)) over (order by ZN) as ZN_MEDIAN,
    min(ZN) over () as MIN_ZN,
    nth_value(INDUS, (select count(*)/2 from boston)) over (order by INDUS) as INDUS_MEDIAN,
    min(INDUS) over () as MIN_INDUS,
    nth_value(CHAS, (select count(*)/2 from boston)) over (order by CHAS) as CHAS_MEDIAN,
    min(CHAS) over () as MIN_CHAS,
    nth_value(NOX, (select count(*)/2 from boston)) over (order by NOX) as NOX_MEDIAN,
    min(NOX) over () as MIN_NOX
    from boston  limit 1 offset (select count(*) from boston) /2
    )
    select
     iif(CRIM_MEDIAN=MIN_CRIM, 'CRIM', ' ') ||
     iif(ZN_MEDIAN=MIN_ZN, 'ZN', ' ') ||
     iif(INDUS_MEDIAN=MIN_INDUS, 'INDUS', ' ') ||
     iif(CHAS_MEDIAN=MIN_CHAS, 'CHAS', ' ') ||
     iif(NOX_MEDIAN=MIN_NOX, 'NOX', ' ') as task_result
    from medians
    """,
    con,
)

,task_result
0,ZN CHAS


Согласно wiki: Если распределение имеет разрывы, то медиана может совпадать с минимальным или максимальным (крайним) возможным значением случайной величины, что противоречит «геометрическому» пониманию этого термина.
[Расчет медианы по stackoverflow](https://stackoverflow.com/questions/15763965/how-can-i-calculate-the-median-of-values-in-sqlite)

4. Выведите разницу между среднем количеством комнат(RM) в домах с самой дорогой стоимостью(MEDV) и 25 самыми дешевыми домами. 
Аналогично по 50, 100, 200, 300 самыми дешевыми домами. (кол-во домов(25,50,100,200,300), среднее кол-во комнат в них, среднее кол-во комнат в самых дорогих, разница).
Напишите влияет ли кол-во комнат на стоимость и как сильно.

In [106]:
pd.read_sql(
    """
    with top_houses as (
        select avg(RM) as avg_rm_top_price from boston where medv=(select max(medv) from boston)
    ),
    numered as (
        select row_number() over (order by MEDV) as rownumber, RM, MEDV
        from boston order by MEDV
     )
     select rownumber as AVG_RM_FOR_LOW_PRICED,
     ((select max(avg_rm_top_price) from top_houses) - avg(RM) over (order by MEDV rows between unbounded preceding and current row)) as DIFF_BETWEEN_TOP_PRICED
     from numered
     where rownumber in (25,50,100,200,300)
    """,
    con,
)

,AVG_RM_FOR_LOW_PRICED,DIFF_BETWEEN_TOP_PRICED
0,25,1.26100
1,50,1.42900
2,100,1.22900
3,200,1.44175
4,300,1.47200


В дорогих домах комнат больше на в среднем 1.36655 комнат (среднее по столбцу diff)

5. Выведите ранги значений колонки LSTAT(процент населения с более низким статусом) в домах с самой дорогой стоимостью (значение LSTAT, стоимость, ранг) среди всех знаечений LSTAT. Напишите какой вывод можно сделать по этим данным.

In [107]:
pd.read_sql(
    """
    with ranks as (
    select LSTAT, MEDV ,RANK() over (order by MEDV) as RANK from boston order by MEDV desc
    ) select * from ranks where MEDV = (select max(MEDV) from boston)
    """,
    con
)

,LSTAT,MEDV,RANK
0,1.73,50.0,491
1,1.92,50.0,491
2,3.32,50.0,491
3,3.70,50.0,491
4,4.45,50.0,491
5,2.97,50.0,491
6,2.88,50.0,491
7,4.63,50.0,491
8,5.12,50.0,491
9,7.44,50.0,491


6. Выведите среднюю стоимость домов граничащих с рекой(CHAS) и нет (граничит/не граничит, стоимость)

In [108]:
pd.read_sql(
    """
    SELECT iif(CHAS=1.0, 'Граничит', 'Не граничит') as chas_human_readable, 
    avg(medv) as average_medv_by_chas from boston group by CHAS
    """,
    con,
)

,chas_human_readable,average_medv_by_chas
0,Не граничит,22.093843
1,Граничит,28.440000


7. Выведите все колонки, у которых среднее значение выше, когда дом граничит с рекой (название колонки) выбирая из CRIM, ZN, INDUS, CHAS, NOX. Напишите какой вывод можно сделать по этим данным.

In [109]:
pd.read_sql(
    """
    with avgs as (
    select chas, avg(crim) as avg_crim, avg(ZN) as avg_zn, avg(INDUS) as avg_indus, avg(NOX) as avg_nox
    from boston group by chas
    ) 
    select  
        iif(lag(avg_crim) over () < avg_crim ,'CRIM', '') ||
        iif(lag(avg_zn) over () < avg_zn,' ZN', '') ||
        iif(lag(avg_indus) over () < avg_indus, ' INDUS', '') ||
        iif(lag(avg_nox) over () < avg_nox,' NOX', '')
        as task_result
    from avgs LIMIT 1 offset 1
    """,
    con,
)

,task_result
0,INDUS NOX


8. Выведите значения долей промышленной застройки(INDUS), концентрации оксидов азота(NOX) и по их перцентилям - 10, 20 ... 100 ( перцетиль(10,20...100),значение INDUS, значение NOX). Напишите прослеживается между ними взаимосвязь

In [110]:
pd.read_sql(
    """
    with ntiled as (
    select  NTILE(10) over (order by indus) as perc_indus,
    NTILE(10) over (order by NOX) as perc_nox, INDUS, NOX from boston
    ),
    percentiles_indus as (
        select perc_indus * 10 as percentile, max(INDUS) as INDUS from ntiled group by percentile
    ),
    percentiles_nox as (
        select perc_nox * 10 as percentile, max(NOX) as NOX from ntiled group by percentile
    )
    select percentiles_nox.percentile, INDUS, NOX from percentiles_indus left join percentiles_nox on 
        percentiles_nox.percentile = percentiles_indus.percentile
    """,
    con,
)

,percentile,INDUS,NOX
0,10,2.89,0.426
1,20,4.39,0.442
2,30,5.96,0.472
3,40,7.38,0.507
4,50,9.69,0.538
5,60,13.89,0.580
6,70,18.10,0.609
7,80,18.10,0.671
8,90,19.58,0.713
9,100,27.74,0.871


Чем выше плотность индустриальной застройки, тем выше концентрация оксидов азота. 